In [2]:
from confluent_kafka import Producer
import socket
import requests
import json
import time

In [3]:
def get_exchanges() -> dict:
    coins_url = (
        'https://api.coingecko.com/api/v3/exchanges'
    )
    content = requests.get(coins_url)
    return json.loads(content.text)

def get_exchange_volume_chart(id: str, days: int) -> dict:
    coins_url = (
        f'https://api.coingecko.com/api/v3/exchanges/{id}/volume_chart?days={days}'
    )
    content = requests.get(coins_url)
    return json.loads(content.text)

In [4]:
all_exchanges = get_exchanges()

In [5]:
def iter_all_exc_volume_charts(delay=10.1, auto_retry=True, retry_delay=31, limit=None):
  for exc in all_exchanges:

    while True:
      volume_one_day_data = get_exchange_volume_chart(
        id=exc['id'],
        days=1
      )

      if auto_retry:
        try:
          volume_one_day_data['status']['error_code']
          print(f'Error! Retrying after {retry_delay} s.')
          time.sleep(retry_delay)
          print('Retrying...')
        except TypeError:
          break
      else:
        break

    time.sleep(delay)
    
    message = {
        exc['name']: volume_one_day_data,
    }

    yield str.encode(json.dumps(message))

    if limit is None:
      pass
    elif limit == 1:
      return
    else:
      limit -= 1

In [6]:
topic = 'coingecko'

producer_config = {
    'bootstrap.servers': 'localhost:9092',
    'client.id': socket.gethostname()
}

producer = Producer(producer_config)

In [8]:
for message_bytes in iter_all_exc_volume_charts():
    producer.produce(
        topic=topic,
        value=message_bytes,
        partition=0,
    )
    print(f'MSG: {message_bytes}')
    
producer.flush()

MSG: b'{"Binance": [[1713370200000.0, "402959.6963868898954187"], [1713370800000.0, "403796.4884875379111667"], [1713371400000.0, "407436.4639985683287132"], [1713372000000.0, "409288.2085165907166062"], [1713372600000.0, "410474.6407778138561486"], [1713373200000.0, "412381.3431948683940869"], [1713373800000.0, "411670.8906028889100786"], [1713374400000.0, "409562.809356951480676"], [1713375000000.0, "409229.2344862155997999"], [1713375600000.0, "405405.9717853983131247"], [1713376200000.0, "403905.2443042633819055"], [1713376800000.0, "401854.3038703057945629"], [1713377400000.0, "400365.1028451523554571"], [1713378000000.0, "399590.4477875935393891"], [1713378600000.0, "399757.190934975643221"], [1713379200000.0, "401719.6207092669458569"], [1713379800000.0, "401765.0472835698286298"], [1713380400000.0, "404521.0179518216826536"], [1713381000000.0, "405653.4669692215136315"], [1713381600000.0, "406989.9632956704475591"], [1713382200000.0, "410169.4732392847875669"], [1713382800000.0

KeyboardInterrupt: 

%3|1713457009.832|FAIL|DESKTOP-6I1TVA2#producer-1| [thrd:localhost:9092/bootstrap]: localhost:9092/1: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1713457010.832|FAIL|DESKTOP-6I1TVA2#producer-1| [thrd:localhost:9092/bootstrap]: localhost:9092/1: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT, 1 identical error(s) suppressed)
%3|1713457040.834|FAIL|DESKTOP-6I1TVA2#producer-1| [thrd:localhost:9092/bootstrap]: localhost:9092/1: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT, 30 identical error(s) suppressed)
%3|1713457070.835|FAIL|DESKTOP-6I1TVA2#producer-1| [thrd:localhost:9092/bootstrap]: localhost:9092/1: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT, 30 identical error(s) suppressed)
%3|1713457100.836|FAIL|DESKTOP-6I1TVA2#producer-1| [thrd:localhost:9092/bootstrap]: localhost:9092/1: Connect to ipv4#127.0.0.1:9092 failed: Co